# Tarea Sesión 1: Estableciendo una línea base

## Implementación

### Paso 1: Cargando la librería y preparando los ficheros

Cargamos la librería `pandas` y guardamos las rutas de los ficheros en sendas variables
En `file` almacenaremos la ruta del fichero de entrenamiento, y en `fileEval` la del fichero de test

In [2]:
import pandas as pd

file = ('./train_en.tsv')

fileEval = ('./dev_en.tsv')

### Paso 2: Cargando los ficheros en dataframes

Cargamos cada fichero en un dataframe, utilizando la función `read_csv()`, indicando la tabulación como caracter delimitador, y filtrando las columnas que queremos utilizar en cada uno.

- En `data` almacenaremos los datos de entrenamiento, filtrando del fichero las columnas "text" y "HS".
- En `evalEtiquetas` almacenamos las etiquetas de test, filtrando la columna "HS" del fichero de test
- En `eval` almacenamos los datos de entrenamiento, filtrando la columna "text" del fichero de test


Tras cargarlos, mostramos sus primeras filas con la función `head()`

In [3]:
data = pd.read_csv(file, delimiter='\t', usecols =["text","HS"])

evalEtiquetas = pd.read_csv(fileEval, delimiter='\t', usecols =["HS"])

eval = pd.read_csv(fileEval, delimiter='\t', usecols =["text"])

data.head()
eval.head()

,text
0,I swear I’m getting to places just in the nick...
1,I’m an immigrant — and Trump is right on immig...
2,#IllegalImmigrants #IllegalAliens #ElectoralSy...
3,@DRUDGE_REPORT We have our own invasion issues...
4,Worker Charged With Sexually Molesting Eight C...


### Paso 3: Realizando el preprocesamiento

Antes de realizar el procesamiento, preparamos los datos para que sean procesables.
Para ello, utilizaremos un idf, un estadístico que nos valora la relevancia de cada palabra dentro del texto

#### Filtrando las etiquetas del conjunto a procesar

Empezamos obteniendo las etiquetas del conjunto de datos. Para ello, en el dataframe de entrenamiento, filtramos la columna "HS"

#### Creando la bolsa de palabras

Utilizamos la librería SciKit Learn, con su clase `TfidfVectorizer` para crear una "bolsa de palabras", valorando la relevancia de cada palabra dentro del texto. Para ello, aplicamos las siguientes opciones:

- La codificación del fichero será 'latin-1'
- Las stopwords se descartarán según el diccionario indicado para el idioma inglés
- Se descartarán las palabras que tengan menos de 5 apariciones (opción `min_df`)
- Se aplicará una normalización de tipo 'l2', Esta norma indica que la suma de los cuadrados de los elementos del vector es 1, y que la similitud del coseno entre dos vectores es su producto elemento a elemento
- Activamos el escalado basado en "frecuencia de términos sublineal" ([sublinear_tf](https://nlp.stanford.edu/IR-book/html/htmledition/sublinear-tf-scaling-1.html)). Este se basa en la premisa de que "20 ocurrencias de una palabra no son 20 veces mas importantes que una sola ocurrencia", dando una fórmula logarítmica para calcular la relevancia de un término. 
- Configuramos un rango de n-gramas de 1 a 3. 

El resultado lo almacenamos en el objeto `tfidf`. 

#### Realizando el aprendizaje

Con la configuración ya establecida, llamamos a `fit_transform()`, pasándole por parámetro el conjunto de entrenamiento. Esta función aprenderá el vocabulario y el idf, y devolverá una matriz de documentos vs términos.

Almacenamos la matriz en el objeto `features`

#### Mostrando los datos

Finalmente, visualizamos el tamaño de la matriz con el atributo `shape`, y su contenido en forma de vector con `toarray()`. Vemos que la matriz tiene 9000 filas y 4380 columnas. 

También mostramos el idf, que nos mostrará un valor de relevancia de cada palabra del texto

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

labels = data.HS

#print(labels)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')

features = tfidf.fit_transform(data.text)

print(features.shape)

print(features.toarray())

print(tfidf.idf_)

(9000, 4380)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5.46070006 7.90786638 8.15918081 ... 8.15918081 6.84699442 8.15918081]


### Paso 4: Dividiendo la colección en subconjuntos

Para evitar problemas de sobreaprendizaje, en los que el clasificador se aprende su propio conjunto de ejemplos, dividimos el conjunto de datos en dos partes: entrenamiento y test. 

Como conjunto inicial, utilizaremos la matriz generada anteriormente, almacenada en `features`. 

Para dividir el conjunto, utilizaremos la función `train_test_split` de SciKit Learn. A esta le pasaremos el conjunto `features`, las etiquetas, y configuraremos varias opciones:

- Los datos se particionarán según sus etiquetas (opción `stratify`)
- Tamaño de los conjuntos de test: 20% del conjunto original
- Semilla para generador aleatorio: 1234

Esta función devolverá una tupla con los 4 subconjuntos generados: 2 de entrenamiento, y 2 de test. 


In [10]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(features, labels,

                                                    stratify=labels,

                                                    test_size=0.2,

                                                    random_state=1234)

#print(X_train)
print(y_train)
#print(labels)

8578    0
5453    1
5601    1
4401    0
2675    0
       ..
7954    0
3453    1
984     1
7460    0
6239    1
Name: HS, Length: 7200, dtype: int64


### Paso 5: Creando el clasificador

Una vez preparados los subconjuntos de entrenamiento y test, preparamos el clasificador.
En este caso, probaremos 3 clasificadores diferentes: 

- *RandomForestClassifier*: basado en un bosque aleatorio. A este le aplicamos los siguientes parámetros:
    
    - `criterion`: Función de medición de la calidad de una partición. Admite dos valores: 'giny' o 'entropy'.
        En este caso, elegimos este último
    
    - `n-estimators`: Número de árboles que conforman el bosque. Establecemos su valor a 100
    - `class-weight`: Peso asociado a cada etiqueta. Establecemos un valor de 0.6 para la clase 1.
    
- *SVC*: basado en una Máquina Vector de Soporte. Aplicamos los siguientes parámetros:

    - `kernel`: Tipo de kernel utilizado por el algoritmo. En nuestro caso, utilizaremos el tipo 'rbf'
    - `gamma`: Coeficiente utilizado por el kernel. Lo establecemos a 0.001
    - `C`: Parámetro de regularización. La anchura de regularización es inversamente proporcional a su valor. Lo establecemos a 300. 
    - `class_weight`: Establece el parámetro C de la clase 'i' a peso_clase[i]*c. En nuestro caso, establecemos el peso de la clase 0 a 0.7

- *MLPClassifier*: basado en una red neuronal. En este caso, dejamos todos los parámetros a sus valores por defecto.





Creamos el objeto clasificador con uno de los tres anteriores

In [12]:
from sklearn.ensemble import RandomForestClassifier

from sklearn import svm

from sklearn.neural_network import MLPClassifier

#clasificador = RandomForestClassifier(criterion='entropy', n_estimators=100, class_weight={1: 0.6})

#clasificador = svm.SVC(kernel='rbf', gamma=0.001, C=300, class_weight={0: 0.7})  

clasificador = MLPClassifier()

### Paso 6: Probando el clasificador con el conjunto de entrenamiento

Con el clasificador seleccionado anteriormente, realizamos el ajuste del modelo sobre el conjunto de entrenamiento

En primer lugar, llamamos al método `fit()` para realizar el ajuste al modelo, usando los conjuntos de entrenamiento. 

Posteriormente, llamamos a `predict()` para obtener las observaciones del conjunto de test X sin etiquetar. Las almacenamos en el objeto `pred_y`. 

Y, finalmente, mostramos la puntuación obtenida sobre los conjuntos de test.

Esta es:

- *MLPClassifier*: 0.713333
- *SVC*: 0.806667
- *RandomForestClassifier*: 0.796111

In [13]:
clasificador.fit(X_train, y_train)

pred_y = clasificador.predict(X_test)

print("CCR: %f"%(clasificador.score(X_test, y_test)))

CCR: 0.709444


### Paso 7: Observando los resultados

Una vez realizado el procesamiento sobre los datos de entrenamiento, observamos los resultados

#### Generando la matriz de confusión

Empezamos generando la matriz de confusión. Esta nos asociará los datos y pronósticos a los fallos y aciertos
Para ello llamamos a la función `confusion_matrix()`, pasándole los conjuntos de test y predicción de Y

Los datos se mostrarán de la siguiente [forma](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html):

- Posición (0,0): Verdaderos negativos (TN)
- Posición (0,1): Falsos positivos (FP)
- Posición (1,0): Falsos negativos (FN)
- Posición (1,1): Verdaderos positivos (TP)

Esta nos da los siguientes resultados para cada clasificador:
    
| Clasificador | TN (aciertos clase 0) | FP (fallos clase 1) | FN (fallos clase 0) | TP (aciertos clase 1) 
| -- | -- | -- | -- | --
| RandomForestClassifier | 918 | 125 | 242 | 515
| SVC | 830 | 205 | 143 | 614
| MLPClassifier | 760 | 283 | 242 | 515


In [19]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred_y)

array([[838, 205],
       [143, 614]])

#### Generando el reporte de clasificación

Además, generamos un reporte de clasificación con varias métricas conocidas: precisión, cobertura, medida-f...

| Clasificador | Precisión clase 0 | Precisión clase 1 | Cobertura clase 0 | Cobertura clase 1 
| -- | -- | -- | -- | -- |
| RandomForestClassifier | 0,85 | 0,75 | 1043 | 757 |
| SVN | 0,85 | 0,75 | 1043 | 757
| MLPClassifier | 0,76 | 0,65 | 1043 | 757



In [21]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_y))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83      1043
           1       0.75      0.81      0.78       757

    accuracy                           0.81      1800
   macro avg       0.80      0.81      0.80      1800
weighted avg       0.81      0.81      0.81      1800



### Paso 7: Probando el clasificador con datos reales

Una vez realizado el procesamiento sobre los datos de entrenamiento, probamos a ejecutarlo con el conjunto de evaluación

Ejecutamos el clasificador y mostramos todas las métricas de evaluación.
Esto nos da los siguientes resultados para cada clasificador:

| Clasificador | TN (aciertos clase 0) | FP (fallos clase 1) | FN (fallos clase 0) | TP (aciertos clase 1) | Precisión clase 0 | Precisión clase 1 | Cobertura clase 0 | Cobertura clase 1 
| -- | -- | -- | -- | -- | -- | -- | -- | -- |
| RandomForestClassifier | 463 | 110 | 138 | 289 | 0,77 | 0,72 | 573 | 427
| SVC | 319 | 194 | 76 | 351 | 0,83 | 0,84 | 573 | 427 
| MLPClassifier | 395 | 178 | 139 | 288 | 0,74 | 0,62 | 573 | 427

También mostramos la tabla de predicción del conjunto, con la clasificación dada a cada ejemplo

In [30]:
evalConTfidf = tfidf.transform(eval.text)

print(evalConTfidf.shape)

evalPredict = clasificador.predict(evalConTfidf)

print("CCR: %f"%(clasificador.score(evalConTfidf, evalEtiquetas)))

print(confusion_matrix(evalEtiquetas, evalPredict))

print(classification_report(evalEtiquetas, evalPredict))

print(evalPredict)

(1000, 4380)
CCR: 0.683000
[[395 178]
 [139 288]]
              precision    recall  f1-score   support

           0       0.74      0.69      0.71       573
           1       0.62      0.67      0.65       427

    accuracy                           0.68      1000
   macro avg       0.68      0.68      0.68      1000
weighted avg       0.69      0.68      0.68      1000

[0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0
 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1
 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0
 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 1
 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1
 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 1 0 1 0 0 0 1 1 1 1 

## Conclusiones

Tras analizar los 3 clasificadores (sin modificar sus parámetros por defecto), observamos que el que da mayor precisión es el SVC. Sin embargo, sus resultados están muy desequilibrados, con muchos mas fallos en la clase 0 que en la 1. Los aciertos se mantienen prácticamente similares en ambas clases.

En los otros dos clasificadores, vemos valores de precisión similares. Aunque, en el MLP, vemos que la precisión de la clase 1 es 10 puntos menor que en la clase 0. En ambos se perciben notables desequilibrios en el número de aciertos, con muchos mas aciertos en la clase 0 que en la 1.

Por esta razón, el clasificador mas recomendable sería el SVC, al ser el mas equilibrado en cuanto a aciertos, y con mayor precisión en ambas clases. Aunque se podría añadir una segunda revisión con el RandomForestClassifier para equilibrar la tasa de aciertos y de fallos en los casos donde el SVC da peores resultados.